In [1]:
from dlinputs import zcom
import threading
import time

In [2]:
def source(addr="zrpush://*:10000"):
    sender = zcom.Connection(addr)
    for i in range(1):
        print "send", i
        sender.send(dict(i=i, a=1, b=2))
    time.sleep(1.0)
    sender.close()
    del sender

In [6]:
thread = threading.Thread(target=source, args=("zrpush://*:10000",))
thread.start()
receiver = zcom.Connection("zrpull://localhost:10000")
result = receiver.recv()
receiver.close()
assert result["a"] == 1
thread.join()

send 0


AssertionError: 

In [7]:
from dlinputs import gopen
thread = threading.Thread(target=source, args=("zrpush://*:10001",))
thread.start()
receiver = gopen.open_source("zrpull://localhost:10001")
result = receiver.next()
assert result["a"] == 1
thread.join()

send 0


AssertionError: 

In [5]:
def source2(addr):
    sender = gopen.open_sink(addr)
    for i in range(1):
        print "send", i
        sender.write(dict(i=i, a=1, b=2))
    time.sleep(1.0)
    sender.close()
    del sender

In [6]:
from dlinputs import gopen
thread = threading.Thread(target=source2, args=("zrpush://*:10002",))
thread.start()
receiver = gopen.open_source("zrpull://localhost:10002")
result = receiver.next()
assert result["a"] == 1
thread.join()

send 0
